# IBM Cloud Hyper Protect Virtual Servers Secure Build and Bring Your Own Image lab (with Docker Hub)

This lab will introduce you to two features of the IBM Cloud Hyper Protect Virtual Servers service:

1. Secure Build
2. Bring Your Own Image (BYOI)

The initial introduction of **_IBM Cloud Hyper Protect Virtual Servers_** (HPVS) offered an Ubuntu image which you could then tailor to your requirements, but if you wanted to replicate your changes onto other HPVS instances, you would have to repeat the custom configurations on each additional HPVS instance you created.  Automation tools like Ansible could help with that process, but wouldn't it be nicer if you could just deploy your own custom-built Docker image, that you only had to configure once, onto an HPVS instance?  That's the convenience that the *BYOI* feature offers. It offers the ability to deploy a digitally signed Docker image into an HPVS instance.

You can deploy Docker images that are digitally signed by you or some other person or organization that you trust. In the case of images you build and sign, the _Secure Build_ feature lets you build, and digitally sign, a Docker image within an HPVS instance running an IBM-provided _secure build server_ that offers the confidentiality, integrity and auditability protections inherent in IBM Cloud Hyper Protect Virtual Servers.  

In this lab you will perform the main use case targeted by these two features working together:

1. You will deploy the secure build server in your own HPVS instance.
2. You will use this secure build server to securely create and digitally sign a Docker image that contains a sample application. For this lab you will use a sample _secure bitcoin wallet application_ provided by IBM for demonstration purposes.
3. You will deploy this Docker image that you've built in a second HPVS instance and then run the sample secure bitcoin wallet application contained within the image. You will configure your HPVS instance to use an **_IBM Cloud Hyper Protect Crypto Services_** (HPCS) instance for key generation.  The HPCS instance provides the security of a _FIPS 140-2 level 4_-certified _Hardware Security Module_, which is the strongest level of FIPS 140-2 certification commercially available.

You will be using an open-source application called _Jupyter Notebook_ to run this lab. Jupyter Notebook allows you to enter commands and see the results of these commands. You are likely reading this within Jupyter Notebook right now.  You will also open a terminal within Jupyter and enter some commands from the terminal. The large majority of the commands will be run from within the Jupyter notebook while a small number of commands will be run from the terminal. Those commands that prompt for user input are run from the terminal, while the vast majority, which don't prompt for user input, are run from within Jupyter Notebook.

## Secure Build overview

The diagram below provides an overview of the major components involved in the secure build process that you will be working with in this lab. An explanation of each numbered annotation is provided below the diagram.

![Secure Build architecture](images/sbsarch.png)

1. The IBM Cloud command line interface (CLI) will be used throughout this lab.  It has already been installed for you in the lab environment.

2. You will be using your IBM Cloud account during the lab.

3. The _secure build_ CLI is a separate CLI used exclusively for the Secure Build process.  You will download this CLI in _Step 5.1_ of this lab and will use it throughout the remainder of the lab.

4. You will be building a container image from an application whose source code is in a GitHub repository. The GitHub repository is specified for you in the configuration file you will build in _Step 5.5_ of this lab.  It is the value of the _GITHUB\_URL_ key given for you in _Step 5.5_. 

5. The container image that you build will be pushed to the _Docker Hub_.

6. The secure build server runs inside a Hyper Protect Virtual Servers instance in IBM Cloud that you will create in _Step 5.11_ of the lab.

7. Your secure build server contains a snapshot of its state.  You can download this _secure build state_ from the server and use it to rebuild the secure build server if necessary.  Although we won't be using this downloaded state in the lab, you will perform the command to download it in _Step 6.10_ of the lab.

**Note:** It is also possible to save the _secure build state_ to _IBM Cloud Object Storage_, but we will not be doing that in this lab.

Okay, let's get started!!

## Initial login to IBM Cloud via the _ibmcloud_ command line interface (CLI)

### Step 1.1 Open a terminal with the Jupyter Launcher

In the browser tab in which you are accessing the Jupyter Notebook to take this lab, you should see a workspace with two tabs:

    1. The tab you are using right now to read this lab.
    2. The Jupyter launcher tab, from which you launched this lab.

Go back to the Jupyter Launcher tab.  Click the *Terminal* tile at the bottom left and the Launcher tab will be replaced with a terminal window that you will use to enter a small number of commands from this lab that require interactive user input.

![Choose Terminal](images/opennewterminaliniksd.png)

When you have done that, come back here to continue with the instructions.

---

### Step 1.2 Use the terminal tab to log in to IBM Cloud

Navigate to your terminal tab, and from there, enter the command `ibmcloud login -r YOUR-REGION --sso`, substituting one of the following regions for YOUR-REGION:  `us-east`, `us-south`, `au-syd` or `eu-de`, which are for United States (East), United States (South), Sydney, Australia, and Frankfurt, Germany, respectively.  

For your convenience the proper command for each region is listed. From the three commands listed below, copy the one applicable for your region, and paste it into the terminal tab and run the command:

`ibmcloud login -r us-east --sso`

`ibmcloud login -r us-south --sso`

`ibmcloud login -r au-syd --sso`

`ibmcloud login -r eu-de --sso`

---

### Step 1.3 Follow the prompts to complete the login

Follow the instructions to log in to your IBM Cloud account.  The login process from the terminal should be like this:

You will first be given a *Y/n* prompt to open the URL to get a one-time passcode, reply *n*. The reason to reply *n* is because in your lab environment your server does not have graphics capabilities and cannot open the URL.

In order to overcome this, you must copy the URL given in the command output and paste it into another browser tab or window. The goal is to obtain a temporary one-time code for you.
In order to get this code to use to log in with the CLI, you may need to log in to the IBM Cloud Web UI with your browser. If so, follow any necessary login prompts. You should eventually be given a page with a one-time code.  Click on that code to put it in your clipboard, and paste it at the input prompt back in your terminal tab.

When successful, your terminal window will look like this, but with your account information of course: 

```

API endpoint:      https://cloud.ibm.com
Region:            us-east
User:              silliman@us.ibm.com
Account:           Barry Silliman's Account (1e963a246cc69a44df65e277e14239d5) <-> 1996902
Resource group:    No resource group targeted, use 'ibmcloud target -g RESOURCE_GROUP'
CF API endpoint:
Org:
Space:
```

When that is complete, come back to this tab and continue in this notebook. Leave the terminal tab open, you will need it later.

---

### Step 1.4 Target the default resource group

---

#### Jupyter Notebook tip

The text you are reading now is in a type of Jupyter notebook cell called _markdown_.  It is intended for instructions, and includes text and possibly images.

Commands that you will enter (with the exception of the small number of commands, such as the `ibmcloud login -r YOUR-REGION --sso` above, that are run in the terminal tab) will be in a second type of Jupyter notebook cell called _code_.  At the top of this page you can see a dropdown box that will show you whether or not you are in a markdown cell or a code cell. The below screen snippet shows where this dropdown box is located:

![Choose Terminal](images/celltypeindication.png)

---

You can see from the output from _Step 1.3_ that no _resource group_ is targeted when you log in. The command in the code cell below will target the _default_ resource group. Resource groups are logical buckets that you can define in order to categorize your IBM Cloud resources as you wish.  We will simply use the _default_ resource group which exists already.

Click on the code cell below that contains the command `ibmcloud target -g default`. You should see _code_ in the dropdown box at the top. 

Now that you have the code cell below highlighted, click the _Run_ button from the menu above—it is the triangle icon—and that command will be run, and its output will be shown.

This will target your _default_ resource group. See the debugging tip below the code cell if this command does not work for you.

In [ ]:
! ibmcloud target -g default

#### Debugging tip

Some IBM Cloud users may not have a _default_ group but instead have a _Default_ group, with an upper case _D_.  If the above command fails, try using *Default* instead of *default*.  You can do that by just clicking in the code cell above and changing the _d_ to _D_ in default.  Then run the code cell again. If that still doesn't work, ask an instructor for help.

---

### A note about running Jupyter Notebook code cells

Observe that to the left of the code cells are square brackets with empty contents. When you run a code cell, an asterisk (*\**) appears between the square brackets.  (If the command runs fast enough you may not even notice the appearance of the asterisk). When the command finishes, a number appears between the square brackets.  During this lab, some of the commands may take several seconds to run, but no command in this lab should take too long. Each time that you run a code cell, ensure that you wait for that code cell to complete execution before continuing with the lab. If the asterisk remains in between the square brackets for more than thirty seconds, ask an instructor for help—none of the commands in this lab should take that long.

The number shown between the square brackets upon command completion is incremented by one each time you run a code cell.  The numbers are helpful in ensuring that you have run each code cell in the correct order.

---

## Create an ssh key for the lab and add it to your GitHub account

### Step 2.1 Create an ssh key pair in your lab system

As part of this lab, you will need to create a key pair that you will then define in your personal GitHub account settings so that this key pair can be used to clone GitHub repositories.

**Note:** If you are an experienced GitHub user and already have ssh keys defined to your GitHub account, you should still create a new key for this lab and remove it from your account when the lab is over, for security reasons. **What is the security concern?** The lab instructors have access to your lab system in order to be able to help with debugging.  If the lab instructors were malicious, they could copy your ssh key, including the private key portion of the key pair.  They are not malicious—they are well-intentioned and kind. But to be completely secure, you should create a new key per the instructions in this step, and then remove it from your GitHub account when you have completed the lab. (The cleanup section of this lab helps you with the key removal from your GitHub account).

Run the code cell below to create a key pair in your lab workspace.

In [ ]:
! ssh-keygen -t rsa -b 4096 -f '/home/jovyan/.ssh/id_rsa' -N ''

---

### Step 2.2 Copy your public key to the clipboard and add it to your GitHub account

The above command created a key pair (a private part that you keep secret and a public part that can be shared safely with the world).  You need to add the public part to your GitHub account settings, so that this key pair can be used to do a clone of the GitHub repository of the sample secure bitcoin wallet application from within the Secure Build Server container.

Run the code cell below to list the public key, and copy this public key into your clipboard:

In [ ]:
! cat ~/.ssh/id_rsa.pub

Copy all of the output from the above `cat` command into the clipboard, and go to [this link](https://github.com/settings/ssh/new) to add this key to your GitHub account. When you get to that page (you may have to log in to GitHub first), put anything you want in the _Title_ field and paste the above output (which is your public key) into the _Key_ field and then click *Add SSH key*.  

**Note:** if you use a password manager and need to use copy and paste to log in to your GitHub account, go ahead and log in and then copy and paste the above output.

---

### Step 2.3 add the GitHub public keys to your _known\_hosts_ file

Run the following code cell which gathers the public SSH keys of github.com and puts them in your lab workspace's _known\_hosts_ file. This is done only to eliminate a prompt about trusting the github.com host in the code cell after this one.

In [ ]:
! ssh-keyscan -H github.com >> "${HOME}/.ssh/known_hosts"

---

### Step 2.4 Verify you have added the key to your GitHub account correctly

Run the code cell below to verify that you have correctly added your SSH key pair to your GitHub account.  You should receive a message that says _Hi \<your GitHub username\>! You've successfully authenticated, but GitHub does not provide shell access._  If you received this message, continue in the lab.  If not, ask an instructor for help.  

In [ ]:
! ssh -T git@github.com

---

### Step 2.5 Create a personal access token for your GitHub account

The build of the sample secure bitcoin wallet application requires that a GitHub personal access token be provided to it. Even if you already have a GitHub personal access token, you are advised to create a new one for this lab. Log in to your GitHub account — you may still be logged in from _Step 2.2_ — and follow steps 1-9 (i.e., you can skip step 10) in [these instructions](https://docs.github.com/en/github/authenticating-to-github/creating-a-personal-access-token#creating-a-token) in the _Creating a token_ section. For step 8, when selecting scopes, you only have to select the first check box that says _repo_. This will also select the subitems in the _repo_ section—leave these subitems checked as well.

Ensure that you save the value of your newly created personal access token, as you will need it later on in the lab.

---

## Create a Docker Hub account if you do not already have one

You will need a Docker Hub account in order to do this lab, because the secure image that you build will be stored in Docker Hub under your account.

### Step 3.1 Sign up for a free Docker Hub account if necessary

If you already have a Docker Hub account, you can skip this step.

Sign up for a free Docker Hub account by going to [https://hub.docker.com/signup](https://hub.docker.com/signup) and following the instuctions.


---

## Create a Docker Access Token

In order to store the image that you will build into your Docker Hub account, you will need a Docker Access Token. This section will guide you through this process.

You could use your Docker Hub account password instead of a Docker Access Token, but this is not a good practice. By providing a Docker Access Token that you create just for this lab, you can delete this token at the end of the lab- thus you will not have to change your password to remain secure.

### Step 4.1 Log in to your Docker Hub account

If you are not yet logged in to your Docker Hub account, click [https://hub.docker.com](https://hub.docker.com) and click the **Sign In** link.
Follow any instructions or prompts necessary to complete the signin process.

### Step 4.2 Create a Docker Access Token

Follow [these instructions](https://docs.docker.com/docker-hub/access-tokens/#create-an-access-token) to create a new Docker Access Token.
Save the value of your access token in a safe place- you will not be able to retrieve the value later.



---

## Create your Secure Build Server HPVS instance

### Step 5.1 Download the Secure Build command line interface (CLI):

In order to set up your own Secure Build Server within an HPVS instance, you will use a command line interface (CLI) that is provided by IBM.  This CLI is implemented in Python and is publicly available (including source code) in a GitHub repository, which you will download now into your lab environment when you run the following code cell:

In [ ]:
! git clone git@github.com:ibm-hyper-protect/secure-build-cli.git

---

### Step 5.2 Change the working directory for the remainder of the lab

Up to now, your notebook has been running from the _/home/jovyan/labdir_ directory. For the remainder of the lab, we want to work from the directory that was just created by the _git clone_ command from the prior code cell.  Our new desired working directory is a subdirectory of our current working directory.

Run the next code cell in order to change directories to the _secure-build-cli_ directory that was created by the `git clone` command that you just ran.

In [ ]:
%cd secure-build-cli

---

### Step 5.3 Install Python packages required by the Secure Build CLI

There are some python packages required by the Secure Build CLI, and running the following code cell will install these packages using *pip3*, the Python 3.x-based package installer:

In [ ]:
! pip3 install -r requirements.txt

---

### Step 5.4 Create the Secure Build Server registration file

Part of the "BYOI" workflow involves creating a _secure registration file_, which contains secrets and metadata that can only be decrypted by the running image.  When you build an image later on in the lab, you will download a secure registration file for the image you will have built, and you will pass this file as an argument to the _ibmcloud hpvs_ command that you will use to create an HPVS instance built from your new image. 

The Secure Build Server itself is deployed securely within an HPVS instance with the help of its own secure registration file.  The code cell below will place this file into your lab environment workspace with the name `secure_build.asc`.

This file is posted publicly on the public documentation page for IBM Cloud Hyper Protect Virtual Servers, but the Secure Build CLI repository on GitHub does not contain it.  That's why you are creating it now.  This file is encrypted, and only the Secure Build Server can decrypt it.  IBM Cloud administrators cannot access your secure build server at all, and you can only access it through the Secure Build CLI, which limits your access to specific functions. Therefore, by design, the decrypted contents of this file cannot be obtained by any human.

**Note:** The code cell below will not produce any output, because it copies the encrypted message into a file.  But you know the command ended if a sequence number is placed within the square brackets to the left of the code cell. 

In [ ]:
%%bash
cat <<EOF > secure_build.asc
-----BEGIN PGP MESSAGE-----
Version: GopenPGP 2.3.0
Comment: https://gopenpgp.org

wcFMAxpkEC0f/4+uAQ//e6gYA9a6JVqAdckH6nLgFyEd1Nqsp3vizOGhvBbh0WbD
QZ2shu7Bz8Cv62Q4vZF2nsNaCEb0nU3umN8rlTGlsJQNA9ar2Hgi3eSdyVIqqtGy
3cDAuLnOEV8pmpU10GAg8UhovNOjY70xnGr1qduaPWqdMtYvfYtV7mrlMlGqEoSV
XeKdlnWhyArgDpukzBZmENPnQ0Myo+XSAvvXpOew4AC6XPlyaJVO45qACFk9M1wR
/BIoqsXwfGefOqYih68wzvV7wbKvOY84sjiburGOfp6vPkw3Hnw1Uhq+GATs+V/9
HxA6h4/1XXbRUQhj64vaEDSSX15tb7xTx+vuRzKEcVIRdUXqSPMD9m74qhajb6Ix
LnJWww6C5pMf+An5ETJaOZ1xCjkXZzBYKH4NSJTHSIEqe8WUqPhoB1QL8J7OMVaI
zvX/InkFsMF0rIm6BTiG/NaMeSfAA1ACl2RXuVTySq0zK3T1M6UVtPXrr9R2AAwF
46aMIiHNVYkqVnarcoxG3u0EcZRzkBqvXonJmR8cTNqLK4iOgv074HMNK0C+WHEr
mdiG8aFg7hvz9RmOq01NKAvMvdOYVzjdg1HnrFPFKZyCz7BH0mTEyJb+07J97ao+
hnm/6b/Rzm2iyuv/FHl5u1MKwUNqAVViNlXRfgxY5U5sBt/b49+e1Q08LFyjF1rS
7QG7jDCNOyWRF/HyHGFCm8BIl7mqcPGXV62xW7ihgSxk40U1jbHCSvf+VulVWEHD
VrcidwFA/sA3oH6BpJbvv9/7DACzhfDBOFY7XBACxneHhovWKgn9XDTSSG8JTpf7
ZstMwJnOt8OFg8Fw6jkZQ0dcBua/W+MisXivBuVA3OeeXaK7Jxew+J98/cF/JC15
+bMqrIs+/RHksmYrgbDxQbHEOV04IPCTm7dp5450NV2ap9179T/L+8IYNL6Ia9VF
g2N2NiFjW11B+1Rihzt7KGH1h/mfzl+InQIlnFjKFaNFJJWlmKEQgcM3sr7cehy4
WaRNUTEtK8/ugtvpdoAfg0kCKIL0hEsj60eineGc2FWJK1fmnSZsr2EdWaMnIVUN
W46CVIpOigd2LjlUu0Or2suuSdwiRGxNfTmJtQHH2i94eYhUbNJPVl6rrAeUGDY/
z+lfJyACKHNx+UtmHXzBxw0LHs52ta/GjGLaLvIXhAA6Qka1NG2yTKDYMO/5IpHz
R4RYdTqrFA+E9wIMxEe688eImh7eB7oGEl1b1XP+l3wOx1XJAR5DTwW2DSLPe+7l
Rfbqo+V/x+CMdWeOGGpvclbR5JMWeixxLdvZLQnhYxt/3HqMBCC908qSjiT7bzo+
0ehYwqOyExjHnh6CTLGJBwo/n+BUkY2Cp8Jzxfn9pU2j0xz6kZRQ98brJ9ngkftx
LUmixIwTRG3Oc5LQxFy9VWeF5mqaRmWcHuVJbFUmpSXb8O8EIwZkT8ePD2gOqgUb
gI16mZDVmxuP0fogS3zI9CsS7I3x5518YORJKV2B4niA9OuU5l6cnofVVNXhLNFs
p/yDEVRQUrE374YRjlX7Gyf9MAb6lQg3vslrT7ZQ5B93tX1ctWmhuHdp7yfieX7n
QtEDqwhyfCU/T5Yi75I6aoEiGVsE7z99LRaExb3DzBM9W/lj0Ut4idjHF8Z+tR6K
+v4Q3AXeeDQHgdeQIObc6fN7592aL/o+JD6VLwfI2LqjWSiPa2LyG3AK6xv6BQMH
oLXw4KdikIOge7W7p+59F1g+Cru6OLUP6YBHeZdtF1CoLurgApNcS5D2WVfQOM2y
hLY87ZuWx35eQR4eCiSC6w/kzBlUxu35Fk5aNLU6t+h01JjnEE6F25LKNUlj5UGp
gyOdMIXZB1nqGWCKGFcanEPvITIb3xndxfiJoTwoqzqWZwHaox9aTKkXltbl9vCe
2+Szzy73BUOeRjhyA3Ey2Okf30zyFz39YSc39sR52mvfj6u/7yYi5275kajEAS9g
BXn4IDsKxCTId7Z2ewi9ETPFGZ2WTrwkinlwempCLW+hg2bdA+BIyubFIB8fepWL
ZVWeQ7qDmmVw5xy1rOmjKuIuJnWZdjsWDD+YOjHbcR0+Eyoe79h5mznsMJvB2M+Q
4OINcfwm3+2D9a6my4/4JnpiTyDHbNPtcejs6LlCVwmifWD99Aa2Z8m/HBqNDIYe
IboptKAwHwmU0UXCGq+pORkpfERHuOcgS0/LhAWz6x0jfa1ux2ehP9p1UCLWcq89
q30FPJA5NIDV28jdVhT8UwP4+RvSsBylU+0FgyFNYpNTjWRKr2G4EhuTpNwu6Z32
lBjKPARk/7Wo3rAHcgsbSbr+5ivrHEN3cWDL8JHAhizRYJFNtRp14FLOuxUhDwjz
JWthTRD4cfUdvStREJla4wIoy7yw0sjwWIAkLbx2govubv+69qFLp9OsesbsONOZ
+rfnNNPvwiYP8FHDOXbK3tz0w6gUAjZaaXLHHzD+7F8BdpC983GK+OuICBheK2Ub
EZurG/KGudZlJRXE8cg5t/8jTWizuRo5fabwOPm+zervlhLugsGUP76LlYM1fGhv
zTu76yJJccN35SjArt+rH33DePyem6Q4Avm+d5IjbDzkbp2/QVx3yZ7RbPWWlUyQ
dBpwPrPfmoJ0iLB014FKrJ8qJ+hE+fea3MmAvbAtx35GPJMwsbLGGh91iNooXaUV
FyDFai6KzpNoWecOjuTttXecpicHYl6V7SimLd3Ji83oeRpz7C2BsnHaew4Z84to
8ANHxBuJkpTw6pNqXSjeTkrwi+VVurJDaA2P2d+eoM841/Qd8k33m5LZKIZ6nBj0
EJRuLbosg/vB2Be59MloWHQdSEDU3qKKcNXC3HCZLrJhVeblHmQVQa/PFP7FPYaO
re58NispyHdp3pis17bBjsztWKWH/a1T4gpEWIdEfnAl1lbdrAjLdPffaWvpPdkg
9R7g8Dfeh6Z0K0VBdF1aBGo2kIrFBYgyI3DwsCSlYLseJTWBBqzEaVbzVNYQHd1F
hZemeweTbjy78U3Q/+zKs6rsrQtCB8cC/dRXhFytJMYgTeo0JhU0Gssk1L6FgSuv
o1ZMMAb2b7yeUzotDKqhq/jC7UAai2kVl1iu5nwGVhbNIRCLWo9QYEOqLuxTejo2
xrzeOejYSzfF11wdG/3thMDrV/3Beyxx3uZtEZKu+zY29xNgYrFBwQTJVcGry6C8
Na6kLNYi6NZp86qQTL80JChZJRszEqX8Kn3+gblbjU+JOjcAAGMaNZuXzCswX2IB
PO3bK4M27N0Iy2/gV4hf8cYjv2z6dzKXoZ2rWIOsBox+Kv/U4roE+OnXXFp4CnAB
uf0OoPWJH8b2+pqtRX0IgU+MOvUuWCehZGk2Q6CoGl65ofIyyT6eesc1x4sNWkY3
+Dxp6YDyA0zZjxS5zPk7jb7kIq8WqJovBITkYQmzLi5bxnxAkrBCBxu96G+MFZ3o
+xBo7kSZdxXFiXzQ8PxnfCEo/B6RvlUcWSPnRfEEmZZvNfxpeYqDGlMP46X13GCn
/sYkdBffyRm+B5wigPjp5guLuNsAWVvyIGus9b6JspgiQBmoJ2yo4OQoFpzJ7aAv
L0a12GIASRromvXVkaVchac4T4IpRREVzODRKkPB6nj1pWR2sbGUdlrNLgaiLNcq
D3B7NdpNQMiJucZ2eq5UTZfzQdX69Q0APRokeNaGhtDGT3lMRIP69DOh1wdxZbJU
mgr741TmBpRiUWMCVD95eqnd8oMER6oSf4RfDslAVffBx82kklPvIXKheZDJAxBY
/ZR+CZyyrS0df/Wefnwg2rDFkMOxU4pNxKFMWjfKaTR2svwhslQLy7hMs8CxZHNW
z4J6EndjNLq4M/KX0yE95V9IUPDYGy0NQRgulvzcTN68Wr3yLhbVN2V6v0TboKbb
5J6M3KuS+3RmhVVsJKstDM65cwPrzORlQ6OLZlPzyWq4xNh4jnjn7RfFDEFUfxeM
WO2MMJk6viJw8HuGKr2k29fspyHIiljoL30f5DkrjlrQukkMYDzGacReINgqArtr
gS7/vsn5gbKHLb/Y2+LjXJcwpqq5fColShPK661HrJszxb2hCujERdXhBb5vo/Qs
o79Uct7anJgy5LC0bU1T1uaeqVOmKbLI3U5Q9uYyt0YfMDjsU+Xhn7pKRie1Mak8
hFpdK/kWQa/CL3pjXVx0xJU5iggNQJXHPeCqsFaDtxz6a34qzmXvc5BKCdXkCkbY
r4Y6oNVFMaMfYx+vhgyg9gKh+6nMOQ83dsghsPcqR4jtlNkMvkWGciFgs/BIAIcG
fF4GAt2EU9MFAcCB8mwNssHIwRB3pUdbpdb6wFpLh/pM7XBp5QRSEYou9uj/8YP+
xpz8vhlejX8Sf1xX+gBEPPPhq9vDTH9SjwMfQDxa5DPdYTPSEpcphD50J5fVBhRX
79IQ68l/rfHv9TH8FDdlbUtCfto+O9Zjl9CDDZr0sUqLtMPXbsNbXOyv41AxuMMU
HPJXDqqDFDF03eb4wqyDpnVU6neVRHvognyU2XCOtMHf8fyvniwUxgjZJiL2rANa
12GNfQI6bmYGiy82UgSSYmD9YGiRB874/xuMILXy1o2+UqzoCRk1sd3hb9D0bpfm
6hrHhhbbuX59ISydUJwhNJo14hMb7E8bbPZTZFy44qvGY8dS33aKj+BZzB0WKnPU
zjnKml5KVDgJQFxiYPB7T7eQdLE0PBZbQVgj5wFb6WZMZgwO89Ub6ZmITNYYK6TQ
F7p1EwhP/ih342gxFaE69vurAf+yz5tP76+eXqTSMHqulxz+BziDZdF9+jeIU7Yw
NVudE6xmFMmLu0PWC9SobCt7tFX7Kg1awaGFJGN1uHcZqCtR3HG3mVRaiqVS8zdA
n4PSiDTeoRcJ+u/Ly+DZYykWui+dziq1lVyYQ4TOhDvrWJmMMGMLw7S6fTgzxVbj
+7jI5pVLMRN7luWcalvLgKPFjCcnVrRHa/+1CQ3GV4au4iVsXkcgkWtZ4R8is8Vq
81s66tPiJerDOQyeY85VwzVyn1nhyX3rR7NMQe9m0xVePeg7t86osTbpl9i8NkCX
/PIhYf8jfNZYesQ/2eVqWmgGJGYZO1yEzcPIySZUSWFdzqj1YCTt673E4R7w6KrD
5WiKOiNIOEL7/RpGqqua9I+TXRL2KzoWXK0BBn6zw02xbGajLZuts46gWzVDbYmq
DenjVRU+ViRWN8RlbZQ/31imhEwZWQPSqpe5wGHH5z5RqHoMxef+IDrgFLrPiUtC
rm7Ve2YwhKEqdWPSgmy2KPY+93ZC6u/GuHAsjqHoGZ3N45D7QrUOl/4SSUnBet2X
9z4CUidhK+cvtQnqZmytEoD/SwdvsTzACs1Ep0IaxVeNDsU8kROKXOdiGM8cbf8I
Qf5fE1f3OAqanoTNjEfZNiWHdVG17lcPgDiQ+1DTG1cwM40zWjd85Cwl6eDpAOQJ
nQdS1Za5PKKZQSUXRNDm+ddCqWkVO5sEXwiCA/CqyQ1mMVZ4+vy6/iGOF9bFym34
DwLZAuom07JwdeRGZioypOogHRl71Uxt8e7BX5V/ftpQmfjcQJHLSGcArJ81QhZB
/9zbnKeAcyxijFtHs5bK34KmLxaCkmmK2MGAW5zLVnUMm5EKvnpBC3g0p/VNx5r9
f4q/pBR//fP0958v9a/MUJiaZBsCzQJpTMwvkQy/f1wu1g+PtFfqK/d19Nt3oqet
b2Df+WfupMjTmg8iM1PvO9CH8Q6WTnIZmGXutc3s+LHZVOY0YHYxt6jWrR7Ir3Fg
X95dOrqBpnZTC/CzaLzY8ydZp0Zm3Dpcq41rMQGEquHJH5ltciBoJ9OSqmUAncXS
jvabsdh+hJjmaCJIC/kCz/OfOmnIr82/ZvEDlvZOwXYLEKqg85UIZcT/WQMPXb+9
JRMfKG4LDiHCmRKCYlBrQRCNyDC2s0ERehnXZTLymk7NlxGENSwELoTzW88knzmP
4Ny2TmPSyUfUa9dHDl++UXn04HN72CT2Ooo7nH4gNf+lJg/6EXYfQ4qfTtvOUBnn
NbEhk9HKvHouNF5v8IG03oZCvyV5VOEifGUu9UYLTptL2ZRSeW0fTDJfgunkEbV8
6zYGDaTDqlq0tgN/bXnKnKhGst04yHm7GLZKu9Mbhs+c5qDZedJNDtS4KMn9DEQ7
NuYnNkzgXkUZaxgppB80b/FXucjl+5oPVcOeDCICOriXODZN+3i8x7iE6VY+a7b6
D3Uq7Ehle2r6AK+ocUAv8UWh2RWDaHI1kALRYB1xkJ2sJnndAYui+jFYgvWAOOoR
8XzD2NmvlMzcSpBP3iBdN+p0CrJ6YXtCQeUhyWe5j2evQK+n1RYEe+eGAWDuFJ68
gjiyMr4QiiDOWnCZX3lzic4CYQVlDPPSOcqFbWFq8nFGaq3OAd9/L9fp3vQw98+e
KPR7ny2oVIyumMIoRWTt+4nch4nhXUg2KcP8p8CPQJY7dgEiAfSQQfTf0H835zIy
lR2FqkZGE+cZNmt6AFrfu7g1T5a0wtzw1JHWnCYPsc/CkRLmuU0GXRCXS3Rztpte
5zA4a6XB0QLd4OI0fFizR/weoaE4FoVlTqy0PUTTLxyifBfWr2ammt0rNmNasMYS
IwPmCCoPX2iRYRosAGrhzZwTw7fLJv78JQAi0piJE3kjZr8MorMQx6LXg9OVsng/
ijr2ftB4ks73EdBuyxYW8rcFu8gMyhSjlCBS8E4kgEDjhhwUTAS7g8KXHBa5xjNB
uGv3s+9Mj0syiq+ZT33lFLF6Ew6/SnRd4OLtgly4e2rjpvKmMg/cGm6XEIMQtklY
l2dzobHftCpKsnrzHm3PnCPu7VcQyEr5jpARs5aJqg8caK8/bCw3JITJuDmgahHC
QqXzSR3I5avQh3Zy964EiRwZgPJopfTql45YqJvj7UqFIYBFSlAMzwqkzu71/u/Y
Xre5gHB3Xpmt74kCVnudYYAWiVZLYzhE7pm202a/s0RNbhE/nbHoknAQmWmJGOp5
bnb+MzsydyyrC5nkP6PuRE9wvG1GFm3Da1hEGVRKOpxAVgezFxwq2wgJl7PXTW1W
SJ6DwOsQHUMamLYmqJRz1TF3nQXh37M8WLG3wqu7P7m9UzUJOGxul2gZozeNaVSu
8auTpv6XGC6T8tWGdykrdcJnSvKbwlUDcTIHjjXSkCPmZERpmAqG1IsOX4NCwz7M
w5IKdRFm5FCoIQSNFdWk8myjItWkvQ6xjINxcQqd4rc+IZnXi7ZyBpgxx88fHIIv
NvPGS/gbsk4WnvMUd7NkYOTU4EwmZpHJI2mP/l39E9he4bCIEnw+PwpHkU7fcrVz
X5Rt9H9y3uLi99UHlNKB6TylnofIjHItEqt7xMBUZD87HdVutLREZ3EbUAo+z+mp
xvZGFaCwKG6re1muiGAxgfUb00pGVYcdvVVqlYBMiESO1Z9leinPTgJJTiuRCJxA
hy+DADVRYWsAtDvwSm5llZjys3qukdZvjOikRkw/05Ra73Oipa+wi80IMUHnDNZq
k+diGGuVCNQUfPVfovgrtigv0XquY9dVl8oROc39HxKVCTdpAKwek0JWquTcK/MH
oi4IcJaBFgpCuJS1uflx7YBgNImGccAS5RfGOA7v2c59jIa4Dqe2iejx2ZK2pc48
1S5JQGwIkSds+10FIm6MP2FtaKMLsE40c7/kaGzragZ0O2/GEyCOjDD6cMkJMGHN
OKsf9sPs4K2JQXx0l5EHBGQSSw2mRBT8Xj5MINCngI1+SaWrOgrvIiOuo8Yahi2i
3i1gSUejlVnxd1/6akPbH7TRlKQlvY4N90yo46JgD1J9alt2P8oRW6POKixMy0/1
Hb3uI/u+7sp/5lYAGYNTSHlK75NKJJtkKpB1vLNt9xj0mNv5zB/3f4bZVDx8tm6I
lLZ8sNYwFqma8Cn45wLBto2Kwx5i54rbdFlFL0YRIpMFJe6paj1LFq4UuRCYSzhN
E32gHm6rsIve+rEfZ6IKsI9THBvLhdMxzRG038Rdd72q4mw43R84ysuV5/EUzzEL
9NZlooWeCNk7/4sIZU+Y97MI5uxjDYdvUECCluicUoofp0QTuaIBOmOwRqA+ceTK
vRISoyfKR5Lte/DAM5/A5d99ASi1HF6+WtfqjgS+HXmbiVAJbrdjWoLvs9x61WHA
ccPCK9QpHsTvWkjNY9FpZvcwvQf2AFC/tANmBeE26AW44A16DAP/QFhFCxs7hcYt
3KjF8aYW1y4nrq6+h+KlDlJ8gEiBEBBzzqrRrUETvbrYOQwvkMQHqvf9vencDZgN
P67RsswTXCNnX4HJlZhsLYK0Qj6sEgYna1hz2FosIxYvEaJO5Me6yTLadB9sNr/g
KJk/3BLLOFKVGWSh9Z696QpSaZiYAZf0xLMYK3zV4dbjUwevlS80BYWC77lXKXsF
Xmg9TL4VQEg3Q7jDpEaoNz5TEs+YsPo8bkf0dSl6TvUEsLm43FVg9MLjJx/swMeM
WK4/Ma/bbMBnkdIomw5qdY0vd5XgN/cyQqR1BanX8zW2sOSAAy3TyMpUOOBZ2IVT
DuiIXo/xQfZRpxlHji50QqPXcdxgBhc8PQy82gl1t+GhV+yoKiJtXekN59xFTkW1
7Idk6NzIdvbHvRnaPd0Im99JzFSUzmkCDXW4yWzMSsG4BD41xSO4WW1E1X9LJxLf
HNwv9sDGDOYLK1w2I0NZWXxkrbagjL5ZQtCQ0ncrhNP8NsOhT1cFN1BKWD9cPpna
9j0ug/ZPplUk4xv/CFXgXfR0JU7ELOTpGaGenigIapZLmYlhn11QIKvVwULHooPi
p/PvA/XzqsvtijXuanmG4W+KQNbkqV5ozWI+wo11lstB09iU4WkjN1b/Z04HEpPk
Fw2Za4NJQ6pjOcxbb9yo8ZojNqdoKaWAmbkWJ9qge/2bw2cGV3yU3oZBDNbLjTuk
OSw8xqtt2WWxw5Ee2CLGy1K1XK5GCeddgFrHoP/Op2p+4QSvgP2Ouer+iPP/OHrc
WjWnaYJeTy1PfmoVR+MO10oM48PSakTlot5hOrMEMaNnSZZHp2/ZDYnxRymYqS+Z
op8eq11UVbQ3t+VkorOPs6ghuW7xt9twdMTPyh6WtvVDFsibxP40BcKL6mBXKVcq
7lWt+ere0hsNKluGcU0oaI8AWThmXCRUGbI54G+zK6H3IBX6d3NujlKvk4y7ZU3g
gWRDaCYKNd7nnXIWkyg6A/zBBsO49IidAsnbxlmuxM/NrnHohOW8DjeH7ZDKkhq3
ewQ0Vl5NTfNa6ZRDy5wj375/Ewn/X0GC/qgSVFvq9Y37a2q9gCedL4t6FFXKYLKe
QV9u1Ofede5qo2Z7Vv6ExWJLH3aUXF8S6BELiExMjx0tE50IqKXDXoektxh4D+i2
8/fMCSOj+iGZPMFCKIsWbkWeQjJkIkGb/hux4zZYhbx6VLZrXTj5eYATCUakzSTu
2aUJDrzROurkeoycEGn6R3aLPHqJGYj27PiBoZAlhKGJgEIMkU2+GXwYVFuBJ4Ln
7OrS5WUJgZRZjRcYynUrhkbWBGCPlll0YMELP5kKu2G9I/lc1FBWm7+xwMlRLPwp
CXynJhGEKdx9ROOrthPHDZAMboUMuKbde1gnTyRmAEfaTr7QlVwff3tOKLYMvSRJ
eLgGWxECaYJcVW2RVHrQOsGXBKOvafYlSBuXxcp1DFWOLUK5ODO+UaI1nddJVJDD
TVoLRWFVW1V54SuzqEfZWqCtiAxz7xVV+TQxGjXT2fEe/VnpPUu9IW77sOZ2aXxK
7KvguVHTjdp1nR3734rDlda8xGn+Q4fTpbm2RJ7ZWIzTa3K8K8kT4eLsK09p+tkU
vP6XQXBRtl79OsRzVagMVfTVIszLaZcnUFwjMmT1DvhNr4bkADoUH2l86sMGntxy
7qpSQ3SDP7sf7rOMl8MgKmeuKwv6+itTXmWHMUOPwChsHLGDifd0j/bcAlFItOjM
d6uW55jS1mgQ9to0tumOTZc7kGiJBFRFQyU0RixlOdxD5TkemOVVhP/lfC3RHdR3
isFuhdSUAfkdE5pFWKKI8752DWpsU2Mw4uC01xnJAlyAByoAiqVvPk6YMRDNxTff
b98GJGJUQD1DlmxqbM1ApVs8fEMZQBEWZdELHsgKO7kJl4wZ3qES9KpyVViDHEId
EfFc8mq3bPPwWaGX/+IgSoih4X1FVALpBekNBey7OKhld+ujsSZ/4CngRgupdyGO
a/vNJA9wrfpbhrFCiwVfh4NCDh9Xx8fXddyKVRGsEuRlnhtugRB5K5tzGOT5pyeL
QNSa86H2+288LdXSu/p1Vix0hsVlD6PErs5cZpVuJws7j8OUjeYFeP3LmWDbwrQW
OSPMcKaCLgx2Taol38rANSvXj/9rM7NdErSEvYeqP8Xph+mngHVV7COFTAQ4nRhx
eKP0pLkHP2WYHa4D1yarz1JmYk2UdrXJRBemo8yXCzVK871DTh5rbA2SXCbhT4Fu
b+mMpoxH4j2R8aoN3ATuizOEmjE0DlYG8F/0XQo4nmZY1jFoyW5uTF4Sv2kycRpE
sTrdfII9zSDAHhYHu477zNQrjQaV/uqjCF6VU6vbN7zkeeg2lw2pdkyhZD21eVE6
QD6jpEeNLqukRRPSObjblEn8Hv2YAP8zRJvgXdX5JdCrVrDAiI/l1sN70B8hekP2
tIAPI2BUE0Hv8G/5jji2Cn1fLsEfyT8SH2LJUDKnJ1zjxKUzN5CHXsFtC31Pem/w
jU2/6gfwJDOGkyuFK+TX5dqHp6z4qWxDv6t3pgCfQ8ujfSR97jJyWxdUmJ4gdYEb
xfR8S3LRYgZjQd4sAb3gS0jtC843IT26M/u7UzCs3PN7J5N6HLyhCEp6AFSlxKw+
1c2/yGYvKRhyuc/UyFF/0jyaeHJ3nLn91gOahKa+gAFfeJq8kHYcj84lVt8y/mux
gmcm/nLpT1lJieQjwEanIWV903ndtwju2aHyGmpY1J0xLn7VNi+mMAiL3SR5ZB6D
uKzofmiXCv2vg4Knnvw0N9y6/xTLHXRcSHxejb7qkjpSrfWvQr8asU+2ILxmpQA7
lKsOo7DdVqFr7Z6gVIvbwukMMoVViHmG/yIF4SiKhWufDjYmmZrG1eghoaSVvJnv
N1wM6VSPNm85LrLnGTMGaRAcVAsDMlX+xmBDXAfr3oJaO1oHfRlZjg6S7En8fi3T
kIkVc4ZmB5KHIoV27IyszN8QPFZ69K4lkVnBGQ5cNJeosfd39cQG2Q2qpuxkF0+/
f/IwqXSig2scx16zDLVuduTIxXTJ7tTSrvuddG8LwgFMf/qLbBOhLDgG/YOG9qMl
0aiDgUkBHi8/imzoGFw2OkNNCpBi1HygJ2C8VF4KbMKzkrYOtqZiJUGHAjMyvQaY
C3EMZC87xGsAobEnNugRU+NoZcTj1UKjrwK6m2ak465xPldp7Gz5YuXg9L2wdqTf
Xob5YIAEDhYuxGKEMuvNv8eKzu0mz+uW7L4nK60pQWnQNYBqxgsPJcdJl9OQMUrF
fv+w+/tT+raEKGTJAbIfA9c+lsFrCIwHbszkuh60DL6u2d12JtpF2qFLN87Dp9vH
Y2fox+6EFe5oepAFtzUoMJ+9jhW/6D+3G5APNKRVMMntYmRpW5xRgdn4NlJ9dkd5
vAnjRhX3girBIFCWUusFPxKzgtHofdNUh03E/tTJaOoq7GFjL0371NmdqNWNND1D
84IMHJTDyLWD46ZKrBnhw7tD9Izz/yOVWHvQtMswgSC+nBoa3UCMpBGBJ+CZIvKv
bo3VK2/YYQVlmPKLMk8aoMi6wAkOBxWnkmNdTtAzr+JrPk4zloMdtzXaAKtR4B1r
vpHhW3ccfw4CuEJJ3YUXRA6bqgFvAacEzBzhhpLO3fEjnNlbEVSVVIMjOghabeiU
wJWtpVUWhTFIIEdoOwQ8fjLLdsWb68kW+2E2luL3QwGkX8X+JcWUupym+p11B/pq
okPjy57tP4KvBwaIfOL/i4N+B3lMRQi5WG1yNpaBrAczLYGFAysXaQ7FflEFAOLD
7Yw6Urz2hPltxupI7jywjJRkPCcp+28eAD0oKr3OdEhPBgJWY8uvk/vWKhAQMaAn
XogjFAlz/CstAhww2G1AYwghjplbJeMnA0Ozceb+HlHLXS12KtZFFjLxg0FrWJJa
jvoaqAGuWpx6kKMA7SR2+cEabfwD252dWphlXoubvdHbZ/bcvJcu/CQgc+y58TYF
uQy5h2BRd5i+em6mm2qLM+T9SYprTL4OKRLaATYfe0Q13E6MFWh4XvtyvV/mkoSy
dL1hnVtpTBNxvB43bdLBvn1kyuMuPJVf/fFv+08Nbq8o4JyRDHwDjxb05RROy/0B
9wDof0OZJyCPaV0coxKyfAFfrqDLzytvCLC0owr/KlqG7lRwLjkfZaTBUdERXu/7
LXJoxf2SihPwX5I0xJRH1g3W40DzzaPyc8UcUTXEcGLDo8wL/7FOxCZUkyFHYkXW
UO7Yvg+eOCEETA6UV60lovXAPkLXf0s/uMbWPyFYxw7KPy/6IvNrFwQo5Mn7DUj/
c48rj6POwJb2ARAbxyS9GNW4GqZlz5lIxeQMgosDeOZBzzjgxcfajRI0JvMoQsXu
tSIDJISF+ELH4YMb3huXCFfrcOgqG4Rju8xi2IWOpwCdWCN4Z24/454HOXQCfIYj
SHNII8J552yTTJjoQxq3RiCoYyTQ4+uv1bolrFInhk5M9pAQmP7tPk1ssNilt2a2
tHXbk7jriQdK80oXsR0XBogWh2ZaiMHS/EGfUK7NmC+GGoNXLii4QvTLx4c77JCg
b5HnpkrfUEF8rtIwwl3hMB0w8IFF+vzR/Fr+lcBwAkTfTwzpOhPGTyRK0XFsqo5U
NXJ/CfOve0q3zvbxUpEqDBYWa9fBmZZBFYhPL4buKkyW8bOgmFkxYd5j2JWbMIQn
tEvTOD/QsjutsN6cDV2ucnTDbWnVXM2rRIvfCPKAKlBYtpl0QYPrA1gR8K8+kBw6
aISP8z/mYxW1FNBX7BTqlnvGSjJ1uAZoGTztwqGCqTnlonnMi8RyIbBFD5AdvDfl
WQbCzmtcGdnizRwvHlZ/QkbMQPs68vMjEOBC1YXv22aZXufHPixOipxYvkpFEvep
YrkCDN1cHpJK3VYaq3csdD+qfvQdF8fEnRUoBBRIAkg4bir+TX/qt8XYWGZsLgMk
Ed7HbegYEtlyOyG8dQ4O47mjxwrwAQDb/icO5aHndGkqvCuxAOZlw3SYwc/bjiQ5
VMkUBmEq6BWlAsXuQmh3MvSQgrgsxwMPqCsHyhb/tb52ruCea+q0wNfQ1M5+Dt6u
b2OSk0ot1NdXr4GOz+MpHX8XoXKasRIi7+eYtzAWjO8Jn3AwGOTq3xAWahVWugx4
RkJJ7vCwj0X7XG8PczJmtBynOqHlWbhWaVRQiVmDdi6ywZsIN0E0LTFDUfk4Zzux
Sju2NaENwOpufeCW8721XdkmgDhdZwP5OQ2mov+A8SmbVVgCmrSjTOk/cLWvBOL2
h5uaqD0sx73NComWqWhEUJ3V+NCNIYXFlcNEGb8SDTfoYr0StcLkybgX/um92lo2
vcFTFBBcziyZXV5188z7hM3FTkrA3ajJC33cSVCqmuypZ5tOS+W+VI8uLZ7uA0QU
4Er2Gkktx7aV02Iq//lz4hZSDqtxSQHfxTQ6kTtxucG+iTOFziOFW3w6icVQRujS
Pk0pbWdOj/ldYiLt4bQHc+0Y95HDxgaXj8aHgcGoKtLh6GiGXx7wWdkwGETeBvVJ
aa9SSZzzRMByQKT3UqIDFU9rDWRaj7m3MZJS2SmSBakJsK7ccpzcCEn6IIG3mql8
TCLzwUytt0gNhD+NxDj9goUq0vuiMpohfu/7jjjfCw8qMN3RNacDKGp6ZTuofFPZ
zh48ZfE2q5Ni8aHgSzOSphytkS6Rpht86Iu7gRLni6F0N4lL8awVW/Kp4G1dgTEy
wdtzr+4fuk4pY17ZX3OK5Mez5oq0TOful3HBWye4XBiYqVFZycJqeePDJ+iIeQXW
r6hgG/kL8+ASF+HCuPLyUVUA+nuX/VF88mRq8QOyS3zPKas/vNuVDFT3/32r1w66
hit/bOK+AYwynvQ0l8H8+CoDQb0IV8AnABm8ynxSywLbwRj0LwAwOjXDwkB5hhp2
7YrO0WdARGMuewCX+/JHg8y51xbXPqxTC551EOYeTBXQzjtJEnoISmUO4JYI3WWk
kmoeBtT8ti17qMkHzjhPnnvbBAvd/5Sfr6z1Zb9zXqC7WvMljChgAkSbYlpaEU24
/aXCexlUHb6LnRguVvxb4JLUS0lGG1mwWsz/joMi6wFP9ZHWTplwtG+/UorW+J6U
i3EDPLwJrDo0wN3erhVWtS7TVB+SkgKd6uHc5fAtXO1BdRl4PdGMn5DMaQ1etFGz
umYsO0ELuwzWUXnmq1Ys2Ev6qiiuPYE3s97CPAyZ1mkb5Hz6NoQ44A/qYHyBinMc
+OXqP4+YHVCBBXPdZSY2UwfyT/s4JXLrKX1ooy5pFfUGvLV5v4yJkiy3SWojsvPm
Y+LoOGJAwj3sc2k8PnTXN9m5ssWV0CZatnH1fopsmO1EmB/E0Uv37XEIW+qXmddw
M69K7h1AECdK4Mm99q2lQloqS5FeN55Z438QXwKDV6tFfyY8PfrhSm5EQwlEajze
mzhgOQlPTRh6kwuIy3WmbAT8+vucQ0ko1N3jGqB/kFIgew5uK2E5M31oYjulwgSc
cMt3vRvmL47ingQ6DqmddWqMrNp6rn1ebxXOaDh0Wwcf9QYPSgtpRLMXVRJmYdEs
62uuI+Ik53cP2UgUg3byzPGLcRY7Bxzu7bEEJU286Ovx+7WFj4tBiSJg9Kdbvcut
Y63Ifs0KEvsQWIBGbkVrUJwe81S6MALazn6EzI+Uw4sIBBMrDqUhMjaElgJfRZrD
0pWQcoJeGX0cUe5aJ5FgcI0f51Aoxp9PxYPJ5pP5/vj56yAXUdABrRwmxsvvQuJ8
71DvHY+3QLrPuAkrRgTyhpj9/Z7UTqowLqFg8vF8IyrNdxHqRZa82JJBpwDxiK5K
J1KRzaexhHmCCaV8rzlLHiFm9aeQx4P4KKp9sKjb7diDq5xwK3hVATpkjagGoHEh
DY5o2b/333ifmwhzc2lI03piejgG+tc7y+I8ZQmuj+4mu5RkA1KVLt5dYnM+UpNH
0RfCCnFZIzf8UBPNJqhBv7Us9C4eOfKZ6SiaENQw6PvKf7OWtQA7uF3Z5hiC3hfP
US6Dgz8YNDLEGqRc5CSeNyqB03TGBpVOy+AKzTY/M6zGcRYC4Jd4r+bp5m6p3R/A
eDO9bq/rmNLGSxHRzM7ZgAcSEJ9jh4W4k4L7R1K5fgnt5bzwUH7Ss+ib4ptw9Qjq
knfH7sVh5VYr/1NQfoAUpl64EZNFgY9L2p7+XPLSLtRIq/2KWFkDK8I/qfJlxOCO
bALdP6xPotfx+6wFz1ufxYsnx6yQ6y6ISGFA+ZgiXiU/ouclvfB4hGp18WbSe62l
T4NbuMANxcB93JnXFjq27m8agINXjLiGE1Bh+SX1BDy/LYDYtD6KKwdVH3+xBBzn
NKqfNKx7QfFgvS+Ltr3jbjn//154yKr1o6YBHtC6hyhDHziYmCxoPZhJIpcL0obW
P5bF8bFbu4C7x+P7W/iv1QyDZJIHk0FbNuah8hfI76rW3Ab05jKbO8NPRn1Y/TaF
06bI6S3Av+ar+UxcwEQAB0Q+9nakmQXssHBKwcWYfhRGLTvvKqAnnAUPkVuov7t3
Eec9PQId7F+9XZSJzaDZW89mwsNsOLHHCwzQPb6cBxeBgvUgVC4NWIS1nVpzavpp
NoAhiY6xWePh8mXpndGHReGQsqL3Cg3f4Q0ihRnMG14pdZGmhoUpw1prBikWqggh
nXTF7CYWJ9EZRf7N/hdQGzJwYxz7wajzojmwLPuUJksBAgVGkCmlwawq0yXbRKue
09lONQ/tTH4Dz93Zpzzk5Mx7/PknVUugAHo9I/C7vpeplbkEmuAxlNAgl33PPRZt
oz6jmdFfJbuer4wc2UjqRcnWmWDh/497PDuUmYb2DhemN97iIBh4PYBFZQv7bI9g
jBmT+cubqw+hmVXDcntGk6UFU6QNIqd5hxQf8Fumul7iRdNO0uj2JTqui8TMOiMr
4WWDjABKEk2QBDdq7u24IXwJEIt56EXAHvQ262c1ypwlZPEXlxB7fELM+nZSO3f4
j0/JKr6QYiR67CT1d8G1nXMh0/TqOhSzbrjwd2RZS5SS1FfdOHb4mp+HCpb5sbrP
kmC+S4O3Q3xu0C3L1EYY5vlkuZCQrnfhdT6gHJXpRdF0r0JvGlVyYUmqIbe8ePO3
WrKJjf5h0toPnne5GmXFzPNxjU6ir/9xEWA+3wAQQZCzUgz+p/kbzE+hYwKDv//7
mgLLmD26byXyZiDei1F/LuxayYN4jIxwnN8H7Rlvc1PREK5TF0QsAKovsLKoCPpb
Nc1yoyjfJHDqxPcpyK8RAYX97LUKvQXSwoKMLaBoyOsJG2SNL3CqYawSxwLY4T5f
wByjUaRY0saQgxHY/6XduIxdi9gvJ5IoUio+eW+gMNlsrStjM4TJ7GBLGu7QLG7p
rbZiLhRy2ZBU+UGntF+K/bMdl7BQwOOrKXfJPj5Wpaq85QaPbzATKYUVHx3KILFn
qj+FpKf+uH0TllVygLDWZT6uMr7697T0YOpZv4SztXByEMWaLttavEKDWDcAdrx6
opV5jmwlEhWDRtO9/sPFGqxcs8rRJCfjN8t/3MWqsn9e7tU0rQKHE+1xp8xUKDmh
5AYN2RolmxikKk0dgy1TTWacYWZjw9Okhg+EGMkZGCRBnggE9DXxmPLHZi5YUNsu
9n5ojFYFivEHVieTKE0VRpSo3QPzcdSmbaOi1qk87T/v02JpDCJv1QRY50V9Wmp1
7wBGxZhq+jgYcW6Zr2T522WjuPH0YV4BVXV13gKRqAD9Lumpv8gWJfU9UI34o8sm
0eXtioaR/Od1peYvf2DGCneMAw9bRTkmAScDpU+8ecslnv/W/6OEPONYzqMhWj86
9zXRJ81PbdQJUUn1DUKheUlEJTIiRc2rJUWO6DpTaUO5EQZs/mA9MrZIwK+0rTIZ
uepT+QWsGsBbZq2S8TfzK+qUS4wJmSNa1RjQptueRDqvvgE3aQhi3uMxTCv9Uqa3
Ci3psnTnAHXe3fJc5HlJmvPYPIwHXi1U4RnTFEWXVeZABUBvPtxJYXTkJ4r/pAUE
oXMu9ZshNeB3e4fGVmJorMlauCdNDq11ecnyVkcYPa0FtC7s3jYai5GbSt56VdA7
FCS63Ir6yrryuRQ5iSVeCqlZj7IfnA4TDG5BCWgwE/PdPrvI+UP0oVT2850cUWtU
AaN8rUY6XK/IofCRQciOGMCrlrTFxb6ySRPwpUmkCvtMwUIuBiXbh69fD1he2aAA
5eiCdJ/81olw6Vk8OBdIB+SNhKWiP5KzGFPJ4LeRIiAJyIjTXrFFYEg+ImLROqAV
41kgcMfKQaTXdRN4qjRI9ee1fMlV4FracJ9d6i6zvUoyS2dclzNVLh/xnPyRhTkB
Fs7i2Bu4PHU/igm2NVAb65MMAMoR2e/0WOPcsV0R6ZKJxpRzCEAxc6Iiv4PEwUir
TqvXR9Qci4q8MbeDgExEuD216qtUKXuKoLORdpYrtHIY8GXWQazYlpTn59yAz3A+
MPVbJ2Ez+ujuyQSbbHZlajb70ZgG3oDyKXM4DT/U9Vw2CjJVO9G/uVIxW6wn0KrR
KOLzAOLpsW6dkT1OEtRG/QJAoB/Lhhwe70r3WsNvFAH2FjX1v265ZFDUyAV2nk8p
3E4asn0bWv/IxdcwR1KmSrK+cKgb9TzXm/GgvWyf/J4yngvACo3I9XHCWF2OPrto
75ccmKjSnDzeFDGMD9nS/PMcSMRG4cxA/1YD6wdfc+RCEJNNzfKaHPUm0lz2OuyV
HFj0PTstMLDydlrrUHSxJvtp9OLTd/3thP9RsRNLN8lyYYNA6IK/M+MFB5PLxsES
T8ZeQBZarQ4haZzTdz6ASwfrlmvR+wLPbdmx41fJ5VlddPToUYRWSjClaeG5v5q9
1A43jOCOM1D833xCsPDDVPrXZszntruxL9unZB9Ozr90qcFjJQ9Q2XPacL7Khbs=
=5nky
-----END PGP MESSAGE-----
EOF


---

### Step 5.5 Create your secure build server custom configuration

Anybody who uses the secure build feature of HPVS will use the same Secure Build Server image, but that image is deployed to that person's individual HPVS instance, and that individual instance is uniquely customized through a configuration file. This ensures that your Secure Build Server can only build the application you specify, and push it only to the registry that you specify in the configuration file which is created by the below code cell.  Don't run it yet!  You have to tailor several settings in the code cell below before you run it:

1. For *both*:

    a. _DOCKER_REPO_

    b. _DOCKER_USER_

 replace `YOUR-DOCKER-HUB-USERID` with your Docker Hub account userid. You either already had an account or you created one in *Step 3.1* of this lab.


2. For _DOCKER_PASSWORD_:

 replace `YOUR-DOCKER-ACCESS-TOKEN` with the value of the Docker Access Token that you created in *Step 4.2* of this lab.


3. For the value of the _ACCESS_TOKEN_ key, which is within the _ARG_ key at the bottom, replace `YOUR-GITHUB-ACCESS-TOKEN` with the value of the GitHub personal access token that you created in _Step 2.5_.

Leave all of the other values unchanged.

Once you have made these substitutions in the below code cell, run the code cell.

**Note:** This code cell also does not produce any output within the Jupyter Notebook for the same reason as the previous code cell.

In [ ]:
%%bash
cat <<EOF > sbs-config.json
{
    "CICD_PUBLIC_IP": "",
    "CICD_PORT": "443",
    "IMAGE_TAG": "1.3.0.5",
    "CONTAINER_NAME": "SBContainer",
    "GITHUB_KEY_FILE": "~/.ssh/id_rsa",
    "GITHUB_URL": "git@github.com:IBM/secure-bitcoin-wallet.git",
    "GITHUB_BRANCH": "master",
    "IMAGE_TAG_PREFIX": "s390x-v1",
    "REPO_ID": "sbs",
    "DOCKER_REPO": "YOUR-DOCKER-HUB-USERID/secure-bitcoin-wallet",
    "DOCKER_USER": "YOUR-DOCKER-HUB-USERID",
    "DOCKER_PASSWORD": "YOUR-DOCKER-ACCESS-TOKEN",
    "DOCKER_CONTENT_TRUST_BASE": "False",
    "DOCKER_CONTENT_TRUST_BASE_SERVER": "",
    "ENV_WHITELIST":  ["ZHSM", "APIKEY"],
    "ARG": {
      "ACCESS_TOKEN": "YOUR-GITHUB-ACCESS-TOKEN"
    }
  }
EOF


---

### Step 5.6 Verify your custom configuration file

You can run the code cell below to display the contents of the file you just created to ensure it looks good.  If you made a typo, you can probably fix it by going back to the above code cell and running it again. 

The code cell below pipes the output of the _cat_ command to a utility called _jq_ (for _JSON query_) which expects well-formed JSON (JavaScript Object Notation) data. If you made a mistake while updating the file that causes a JSON syntax error, the _jq_ utility will detect it and print an error message.  

If you put in a wrong value that doesn't cause a JSON syntax error, _jq_ will not catch that, so inspect the output of the code cell carefully.  All variables that you are required to change started with _YOUR_, so it isn't a bad idea to look for the word YOUR in the output—you shouldn't find it!

In [ ]:
! cat sbs-config.json | jq .

---

### Step 5.7 Generate client certification authority and client certificate

Secure communication between you and your secure build server is achieved by using mutual Transport Layer Security (TLS). TLS is the same security protocol that protects most Web traffic today, but most Web interactions only require the server to be authenticated.  Mutual TLS refers to the situation where not only does the client want to ensure that the server is legitimate, but the server also wants to authenticate the identity of the client.  Your Web browser is constantly receiving server certificates and verifying them while you surf the web. Client certificates are what are used when the server wants to verify the client.  

Create the client certificate and client certification authority (CA) by running the code cell below. You should receive two lines of output, with the second line being `INFO:root:client_certificate: generating client CA and certificate`

In [ ]:
! ./build.py create-client-cert --env sbs-config.json

---

### Step 5.8 Generate server certificate and server key

Create the server certificate and server key by running the code cell below. You should receive four lines of output, with the fourth line being `INFO:root:server_certificate: Successfully generated server certificate`

---

### Step 5.9 Display your environment which shows your client certificate and CA and your server certificate and key

The next code cell will display the client certificate, the client certification authority, the server certificate and the server key, all in base64-encoding.

**Note:** Keep the output from the below code cell.  You will be copying most of its output into your clipboard in a subsequent step.

If you already know about _base64-encoding_, go ahead and run the code cell.  If you don't, read the next paragraph if you're curious about what base64-encoding is.  

Base64-encoding is a means of encoding any data, including binary data, into human-readable characters.  It takes groups of 6 bits, converts these six bits into a number from 0 to 63  (2 to the 6th power is 64), and replaces these 6 bits with one of 64 human-readable characters.  Which 64 human-readable characters were chosen?  Lowercase alphabetic characters a-z for 26, uppercase alphabetic characters A-Z for 26 more, the digits 0-9 for 10 more, which gives us 62.  Then `+` and `/` were chosen to make 64!  It should be noted that base64-encoding should not be confused with encryption.  It is easy to convert a base64-encoded string back to its original format.


In [ ]:
! ./build.py instance-env --env sbs-config.json

---

### Step 5.10 List your current HPVS instances

Run the following code cell to list your current HPVS instances. If you are new to HPVS, you won't have any.

This lab uses the free version of HPVS instances, and IBM Cloud users are limited to two free instances at a time, and the lab requires two instances. If you are an experienced HPVS user and you have paid HPVS instances, they won't be affected by this lab.  If you are currently using one free HPVS instance and don't want to delete it, talk to your instructor who can help you work around this situation. If you are currently using both free HPVS instances and can't delete at least one of them, then unfortunately this lab will not work unless you are willing to incur the costs of creating chargeable HPVS instances for this lab.

In [ ]:
! ibmcloud hpvs instances

---

### Step 5.11  Create your Secure Build Server with the _ibmcloud hpvs_ CLI

The code cell below has the first part of the `ibmcloud hpvs instance-create` command filled in properly.

**You need to make two changes.**

**First**, you need to change _YOUR-LOCATION_ to one of the locations as explained in the following paragraphs.

If you logged in to _us-east_, then choose one of these three locations: `wdc04` or `wdc06` or `wdc07`

If you logged in to _us-south_, then choose one of these three locations: `dal10` or `dal12` or `dal13`

If you logged in to _au-syd_, then choose one of these three locations: `syd01` or `syd04` or `syd05`

If you logged in to _eu-de_, then choose one of these three locations: `fra02` or `fra04` or `fra05`

**Second**, you need to add arguments for the `CLIENT_CRT`, `CLIENT_CA`, `SERVER_CRT` and `SERVER_KEY` variables.  All you have to do is copy them from the output from two code cells above, from _Step 5.8_.  Copy the four variables by selecting the output starting from _-e CLIENT_CRT_ and extending to the end of the output. In other words, you are copying almost all of the output from _Step 5.8_ except for the few lines of output that precede the start of the first *-e CLIENT_...* variable that is listed.

I have left a blank line in the code cell below in which you can paste these four excruciatingly long environment variables. Click in the code cell so that your cursor is at the beginning of the blank line 2 of the code cell, and then paste in the variables at that spot.

**Tip:** When trying to copy this long string in the clipboard, when the entire string isn't visible on the screen at once, I have had smoother scrolling when starting from the bottom and then sweeping up to the top to select the text.  Your experience may differ.

Once you have tailored the code cell per these instructions, run it.  This command may take several seconds to run.

In [ ]:
! ibmcloud hpvs instance-create SBContainer free YOUR-LOCATION --rd-path "secure_build.asc" -i 1.3.0.5 \

# copy and paste, in the blank line above me, the super long -e CLIENT_CRT=...  -e CLIENT_CA=...  -e SERVER_CRT=...  -e SERVER_KEY=...  from the output of the command \ 
# a couple cells above.  you can leave me here because I'm just a comment.

---

### Step 5.12 Check the status of your Secure Build Server HPVS instance

It will take several minutes for your instance to be built. The command output from the previous code cell in *Step 5.11* conveniently lists the _ibmcloud hpvs_ command you can use to display the status. Copy the entire `ibmcloud hpvs instance crn:...` command above, which has your unique instance id (shown in the output of the above command, it starts with _crn:_) into the code cell below and run it periodically (about once a minute) until the instance is successfully created. 

**Tip:** Your instance is successfully created when the value of the _Cloud state_ field of the output is _active_. For the first several minutes you will see that the _Cloud state_ is _provisioning_ and that most of the values listed below _Cloud state_ are _not available_.  At a certain point, you will see that those values that were _not available_ have been populated, but you may see that the _Cloud state_ remains as _provisioning_ for a little while. Ensure that you wait for the _Cloud state_ to change to _active_ before proceeding with the lab.

In [ ]:
! 
# copy and paste above the entire 'ibmcloud hpvs instance crn:.....' commmand from the \
# output from the previous command, and run it periodically (every minute or so) until 'Cloud state' is active.

---

## Create your sample Secure Bitcoin Wallet Application BYOI image ##

### Step 6.1 View your custom configuration file again

The code cell below will show the current state of your secure build configuration file.  You have already looked at this file from an earlier code cell, but we're repeating it here so that you can observe the key thing I want you to note from the output—that the IP address value of the _CICD_PUBLIC_IP_ key is blank.

In [ ]:
! cat sbs-config.json | jq .

---

### Step 6.2 Add the public IP address of your secure build server to your custom configuration file

Now that your secure build server instance is running, make a note or copy the IP address shown in the *Public IP address* field from the command output from *Step 5.12*. The code cell below uses an awkward-looking, yet simple _sed_ command to add the IP address in the proper spot.  

**Editing the below code cell is required:** Replace the _xx.xx.xx.xx_ string below with the Public IP address of your secure build container, then run the code cell. Make sure you leave the double-quotes surrounding _xx.xx.xx.xx_ in place. You should not see any output from this code cell:

In [ ]:
! sed -i -e 's/\"CICD_PUBLIC_IP\"\: \"\"/\"CICD_PUBLIC_IP\"\: \"xx.xx.xx.xx\"/' sbs-config.json

---

### Step 6.3 Verify that you added your public IP address correctly

Run the code cell below to repeat the command that displays your configuration file. Ensure that your public IP address was added properly.  Instead of _"CICD_PUBLIC_IP": "",_ the line should look like _"CICD_PUBLIC_IP": "xx.xx.xx.xx",_ but with your Secure Build Server instance's public IP address instead of _xx.xx.xx.xx_.

In [ ]:
! cat sbs-config.json | jq .

---

### Step 6.4 Update your /etc/hosts files

You need to update the _/etc/hosts_ file in your lab workspace in order to map your secure build server's Public IP to the hostname of `sbs.example.com`. You will want to add a line that looks like this, substituting your instance's Public IP address for `xx.xx.xx.xx` in the below example:
_xx.xx.xx.xx    sbs.example.com_
Run the following command in your terminal tab to get to an editor and update your /etc/hosts file. Ask an instructor for help if necessary if you are not familiar with the _vi_ editor:
_sudo vi /etc/hosts_


---

### Step 6.5 Check the status of your Secure Build Server

Run the following command to check the status of your Secure Build Server instance. The value of the _status_ field in the output should be empty, i.e., `"status": ""`:

In [ ]:
! ./build.py status --env sbs-config.json

---

### Step 6.6 Initialize the configuration for your Secure Build Server instance

Run the following code cell to initialize the configuration for your Secure Build Server instance. The status of the output should be `OK`:

In [ ]:
! ./build.py init --env sbs-config.json

---

### Step 6.7 Start the build of your sample secure bitcoin wallet application image

The code cell below will start the build of your application image. The command returns quickly, as soon as it submits the request to start the build.  Your status message from the command output should say `OK: async build started`, which indicates that the build of your image has been started up on your Secure Build Server container. The command does not wait for the build to complete. The build will take about twenty minutes. Try to check out the next two steps before you take a break.

Run the below code cell to start the your build:

In [ ]:
! ./build.py build --env sbs-config.json

---

### Step 6.8 Check the status of your application image build

Check the status of your build by running the code cell below. You can check it periodically—your build should take about twenty minutes to complete. You can run this code cell periodically, as well as the command in the subsequent code cell (_Step 6.9_) which displays the log messages from the build.  For now, check the status by running the code cell below.  Your build has completed successfully once the output of the code cell has the value of *success* in the *status* field:

In [ ]:
! ./build.py status --env sbs-config.json

In the above code cell, until the build completes, you may see several different statuses, including:

_cleaned up_   This status can appear at the beginning of the build but you may not see it as the status quickly changes to _github cloned_.

_github cloned_  This status appears most of the time as this status remains until the image build is complete, and this takes the largest amount of the entire build time.

_image built_  Once the image is built, but before the push of the image into your Docker Hub account is complete, you may see this status.

_image pushed_ Once the image has been pushed to your Docker Hub account, you may see this status, but usually the build completes very soon afterwards, so you may not see this status, as it should hopefully quickly change to _success_ status.

_success_ Once the secure build process has completed successfully, you will see this status. Once you see this status, you are free to move on (Step 6.10). Remember, you can run the command to display the build log periodically (Step 6.9) while you are waiting for this _success_ status. 

---

### Step 6.9 View the log from your application image build

Your secure image build will produce a lot of output in its log. 

You can check the log of your build by running the below code cell. 

**Jupyter Notebook tip 1:** In some environments this output may be placed in a scrollable window, but in other environments all of the log output may be placed into the notebook, which makes navigating to the rest of the Notebook more difficult.  If this happens to you, you can enable scrolling by clicking in the output, then right-click and choose _Enable Scrolling for Outputs_. This will place the output into a scrollable window. 

**Jupyter Notebook tip 2:** Additionally, you can clear the output of this command from your Jupyter Notebook by clicking within the output, then right-click and choose _Clear Outputs_.  (The build logs are retained on the Secure Build Server and can be retrieved at any time).

**Jupyter Notebook tip 3:** Running a code cell clears the output from any previous execution of the code cell, so if you want to view the log more than once, you don't need to clear the output as described in the tip given in the preceding paragraph.

In [ ]:
! ./build.py log --log build --env sbs-config.json

---

### Step 6.10 Download the state image of your Secure Build Server

Run the code cell below to download the state image, which is a backup of the state of your Secure Build Server instance. This can be used to recreate your Secure Build Server if necessary. This backup also contains the logs of any builds you run on your Secure Build Server.

In [ ]:
! ./build.py get-state-image --env sbs-config.json

---

### Step 6.11 Verify the manifest file from your build

The Secure Build Server creates a signed manifest file for each successful build. The manifest file is useful for audit purposes. You can verify the source and integrity of the build and the built image or you can pass the manifest file to an auditor to do so. Run the below code cell to download and verify the manifest file:

In [ ]:
! ./build.py get-manifest --env sbs-config.json  --verify-manifest

The above command downloads the manifest file and verifies the digital signature on it.  In the last line of output from the above code cell you should see *verify=OK* which indicates that the digital signature is valid and is proof that the manifest file has not been tampered with.

---

## Create HPVS instance using the BYOI image that you just built

### Step 7.1 Download the encrypted registration file for the image that you just built

In order to create an HPVS instance using the application image you just built, you must download an encrypted registration file for your application image that was created by the build. This registration file can only be decrypted, and thus used, by your application container image.

**Note:** This command won't work if run in a code cell because of the need to type in a passphrase. That is why you have to run it in the _Terminal_ tab.

Copy the command below and switch to your _Terminal_ tab and run the command, and enter a passphrase of your choosing when prompted.

`cd /home/jovyan/labdir/secure-build-cli && ./build.py get-config-json --env sbs-config.json --key-id secure-build-941893e-1`

You will be promted to enter a passphrase three times—the first two times to set the passphrase on a new key, and then this new key is used to sign the registration file, so you are asked to enter the passphrase you just set. For this lab, you only need to remember it for the duration of this command—it's not used in any subsequent commands in the lab. If you enter a simple passphrase, you may be warned about this, but it's okay to accept it anyway for this lab, if you promise not to be so lackadaisical in the real world!

Note the value chosen for the *--key-id* argument:  *secure-build-941893e-1*. 
If you look at your command output a few code cells above, from _Step 6.11_, where you checked on your build status, you can see _941893e_ at the end of the value of the *image_tag* key, i.e.,  `"image_tag" : "s390x-v1-941893e"`. 
This value _941893e_ was set by the secure build server. It is not unreasonable for you to wonder why the secure build server chose this particular value. It would be unreasonable for me to fail to provide an answer now that I've piqued your curiosity. That value is the beginning of the hash of the most recent commit of the branch of the GitHub repository that you specified in the secure build server configuration file that you created in *Step 5.5*. Check it out for yourself at [https://github.com/IBM/secure-bitcoin-wallet](https://github.com/IBM/secure-bitcoin-wallet). The secure build server appends this to _s390x-v1-_ in order to form a tag name for the image it has built. 

The *--key-id* value can be anything, but putting this hash value as part of the name helps to keep from confusing different keys if you have multiple keys.



---

### Step 7.2 Launch your BYOI application as an HPVS instance

Make the change described here to the below code cell and then run the code cell to create your HPVS instance using your BYOI image that you just built.  The value for the `-i` argument must be the value of the *image_tag* key in the output of *Step 6.11* which checked your build status.

**Tailoring the below code cell before your run it:** You need to make two changes:

1. You need to change _YOUR-LOCATION_ to one of the locations as explained in the following paragraphs.

    If you logged in to _us-east_, then choose one of these three locations: `wdc04` or `wdc06` or `wdc07`

    If you logged in to _au-syd_, then choose one of these three locations: `syd01` or `syd04` or `syd05`

    If you logged in to _eu-de_, then choose one of these three locations: `fra02` or `fra04` or `fra05`
    

2. Ask your instructor for the APIKEY which allows access to the instructor-provided Hyper Protect Crypto Services instance, and replace _YOUR-INSTRUCTOR-PROVIDED-APIKEY_ with the value that the instructor gives you.

In [ ]:
! ibmcloud hpvs instance-create securewallet free YOUR-LOCATION --rd-path sbs.enc -i s390x-v1-941893e \
-e "ZHSM"="ep11.us-south.hs-crypto.cloud.ibm.com:8367" \
-e "APIKEY"="YOUR-INSTRUCTOR-PROVIDED-APIKEY"

---

### Step 7.3 Check the status of your BYOI application instance

It will take several minutes for your instance to be built. The command output from the previous code cell in *Step 7.2* conveniently lists the _ibmcloud hpvs_ command you can use to display the status. Copy the entire `ibmcloud hpvs instance crn:...` command above, which has your unique instance id (shown in the output of the above command, it starts with _crn:_), into the code cell below and run it periodically (about once a minute) until the instance is successfully created. 

**Tip:** Your instance is successfully created when the value of the _Cloud state_ field of the output is _active_. For the first several minutes you will see that the _Cloud state_ is _provisioning_ and that most of the values listed below _Cloud state_ are _not available_.  At a certain point, you will see that those values that were _not available_ have been populated, but you may see that the _Cloud state_ remains as _provisioning_ for a little while. Ensure that you wait for the _Cloud state_ to change to _active_ before proceeding with the lab.

In [ ]:
! 
# copy and paste above the entire 'ibmcloud hpvs instance crn:.....' commmand from the \
# output from the previous command, and run it periodically (every minute or so) until it is running.

---

### Step 7.4 Edit this markdown cell to create a clickable link to get to your sample application

Once your instance is active, note the value of the _Public IP address_ field from the output from _Step 7.3_.  

Double-click anywhere within this paragraph to go into _edit_ mode of this markdown cell.  Then, in the final line of this markdown cell, change `xx.xx.xx.xx` to your public IP address.  The actual URL is on the right side of the line (between parentheses). The text of the link is on the left side of the line in between square brackets.  You may as well change both occurrences of `xx.xx.xx.xx` for clarity, but the one that matters is the link on the right side.   (If you don't see both sides you haven't succesfully got into edit mode—keep trying or ask an instructor for help).  After you have made the change, run this markdown cell by clicking the *Run* icon in the Jupyter Notebook menu above. This will render the markdown and present you with the properly formatted link. Click the link and you will bring up your sample secure bitcoin wallet application.

[Change the xx.xx.xx.xx to your public IP and then click here](https://xx.xx.xx.xx/electrum)

## Running the Secure Bitcoin Wallet Application in your BYOI HPVS instance

### Overview

This application is a secure [Electrum](https://electrum.org/#home) Bitcoin wallet found at [this GitHub repository](https://github.com/IBM/secure-bitcoin-wallet). You can use it to store the identities necessary to transact on the Bitcoin public blockchain network. While this version of the wallet is just a proof of concept, running this type of wallet on Hyper Protect enables you to protect your digital assets from bad actors such as malicious server administrators.

*This Electrum Bitcoin wallet functions regularly, while inheriting the security benefits of the Hyper Protect Virtual Server platform. This ensures that a legitimate version of the application is running (Secure Image Build process) and that administrators cannot compromise the application (Hyper Protect Virtual Server deployment).*

Here is an overview diagram of the _sample secure bitcoin wallet application_ that you built in this lab: 

![Secure Bitcoin Overview](images/sampleapparch.png)

The application, by default, uses software cryptography in order to generate keys, but it can also be configured to use _Hyper Protect Crypto Services_ in order to generate keys more securely, which is what you have done earlier in this lab when you specified the _ZHSM_ and the _APIKEY_ environment variables in _Step 7.2_.


### Step 8.1 Account Registration

Registering a fake account for lab purposes.

1. Choose `Register` since you won't have an existing account on the newly deployed server.

    ![Secure Bitcoin Wallet Welcome](Bitcoin_Wallet_Images/Bitcoin_Wallet_Homepage.png)

2. Enter the registration details you desire.

    ![User Registration](Bitcoin_Wallet_Images/Wallet_register_info.png)


*Do not worry about the information you enter on the registration page (shown below) as it won't map to anything critical to the real world.  You just need your password to log back in if you log out of the application.*

### Step 8.2 Wallet initialization

It's time to set up your Secure Electrum Bitcoin wallet itself!

1. Bring up the wallet screen

    ![Wallet Start](Bitcoin_Wallet_Images/Click_Wallet_Start.png)

2. Enter in a `password` for your wallet and click `Create Wallet`

    ![Create Wallet](Bitcoin_Wallet_Images/Create_Wallet.png)

3. Load your newly created wallet to enable it

    ![Load Wallet](Bitcoin_Wallet_Images/Load_Wallet.png)

4. Reload the page

    ![Initialized Wallet](Bitcoin_Wallet_Images/Reload_new_wallet.png)

5. Your wallet is ready for use in all of its glory

    ![Locked and Loaded Wallet](Bitcoin_Wallet_Images/Loaded_Wallet_Time.png)

### Step 8.3 Receiving Bitcoins

A wallet without currency is blasphemy, so it's time to fill it on up with fake bitcoins.

1. Go to the `Receive` section of the wallet and get the bitcoin wallet address.  

    ![Wallet receiving bitcoin](Bitcoin_Wallet_Images/Receiving_bitcoin.png)

    *This is the address whose alloted bitcoins you lay claim to.*

2. Open [this bitcoin test faucet site](https://bitcoinfaucet.uo1.net/send.php) [click on the link] and use it to transfer bitcoin value to the `Receiving address` in your wallet **from step 1 above**.

    ![Transfer Bitcoins](Bitcoin_Wallet_Images/Send_bitcoins_to_wallet.png)

    *Instead of transacting on the "real" bitcoin blockchain network, we are using a [testnet](https://medium.com/myetherwallet/understanding-blockchain-changes-testnets-and-mainnets-c2171a8e835f). This means that the network participants agree that the assets on the network have no value. Testnets are used for testing changes to the network, testing applications and introducing new users to the technical aspects of network. There are testnets for many of the public blockchain networks including of course bitcoin and ethereum. Test faucets (like the one we are using) act as distributors of tokens to other users on the testnet.*

3. Switch back to your wallet tab and go to the `History` section and then reload the page.

    ![Check History](Bitcoin_Wallet_Images/Check_History_Receive_Transaction.png)

4. You should now see your wallet address as the recipient of a `pending transaction` from the bitcoin testnet faucet.

    ![First Transaction History](Bitcoin_Wallet_Images/Transaction_History_no_confirmations.png)

5. After clicking for more information you get a detailed transaction overview

    ![Receive Transaction 0 Confirmations](Bitcoin_Wallet_Images/0_confirmations_block_details.png)

    *The number of [confirmations](https://en.bitcoin.it/wiki/Confirmation) a block receives is the number of blocks deep the block that has included the blockchain transaction is. In other words, `1 confirmation` means that the current head block includes the transaction. `2 confirmations` means that the current head block's parent block contains the transaction and so on. The deeper the transaction is (more confirmations it has) the more mining power it would take to "reverse history" and undo the transaction.*

    Since there are `0 confirmations` in the picture above, it means that the transaction is not yet included in a block. This is why it has no timestamp yet and its block number is listed as `0`.

6. After waiting anywhere from a minute to a few minutes (depending on the current testnet traffic) you should see a confirmation on your transaction.

    *You will need to keep reloading the page for it to update. It doesn't update automatically.*

    ![1 Confirmation Received](Bitcoin_Wallet_Images/1_Confirmation_Received.png)

    There is now a timestamp and a block number!

    *As time passes more confirmations will accumulate. You can check on the `confirmations` again at the end of the lab to show yourself this (Given enough time has passed for one or more additional confirmations to occur).*

### Step 8.4 Returning Bitcoins

Since you are feeling charitable you decide to transfer your testnet value back to the faucet's address.

1. Go back to [the faucet site](https://bitcoinfaucet.uo1.net/send.php) and get its bitcoin address

    ![Faucet Address](Bitcoin_Wallet_Images/testnet_faucet_address.png)

    *This address is here for easy copy and paste (ensure this matches what the application shows in the browser before using this, as the bitcoin address they use for returning funds changes occasionally):

    ``` bash
    tb1ql7w62elx9ucw4pj5lgw4l028hmuw80sndtntxt
    ```


2. Send bitcoins to faucet using `Send` section of wallet

    ![Send Bitcoins to Faucet](Bitcoin_Wallet_Images/Send_bitcoins_back_to_faucet.png)

3. Once sent you will see the `Signed transaction` information filled out on your wallet page.

    ![Bitcoins Sent Back to Faucet](Bitcoin_Wallet_Images/bitcoin_send_transaction_output.png)

4. You can go back to the [bitcoin faucet site](https://bitcoinfaucet.uo1.net/send.php) (you may have to refresh the page) and see the transaction as green there with a sender of your address and a transaction ID matching the one you sent.

    ![Bitcoins returned to faucet](Bitcoin_Wallet_Images/transaction_giving_coins_back_to_faucet.png)

5. You can view both the initial received transaction and the newly sent transaction in the `History` tab of your bitcoin wallet.

    *You have to keep reloading the page for it to update with new confirmations. If you wait long enough, eventually these transactions will gain confirmations as in the picture below*

    ![Both transactions with 1 confirmation](Bitcoin_Wallet_Images/both_confirmed_transactions.png)

6. You can continue to play around with the electrum wallet before moving on.

### Summary

You have participated in the bitcoin testnet using a securely built electrum bitcoin wallet powered by a Hyper Protect Virtual Server. Congratulations!!! 

Now, it's time to clean up..

---

## Clean up when you are done with the lab

This section describes important cleanup steps to perform when you have completed the lab. It will guide you through deleting the following artifacts that you created during the lab:

1. The Secure Build Server HPVS instance you created
2. The sample Secure Bitcoin Wallet Application HPVS instance you created
3. The Secure Bitcoin Wallet Application container image that you created for this lab
4. The Docker Access Token that you created for this lab
5. The GitHub ssh key you created for this lab
6. The GitHub personal access token you created for this lab


---

### Step 9.1 List your hpvs instances

Run the code cell below to list your HPVS instances:

In [ ]:
! ibmcloud hpvs instances

---

### Step 9.2 Delete your Secure Build Server HPVS instance

In the output from _Step 9.1_, find your instance named *SBContainer*, and copy its CRN (_Cloud Resource Name_). Paste it at the end of the `ibmcloud hpvs instance-delete ` command in the below code cell and then run it.

**Note:** Make sure that your paste operation leaves a space between the end of your instance's CRN and the _#_ character that begins the comment in the code cell. 

In [ ]:
! ibmcloud hpvs instance-delete --force     # <- paste your full CRN name to the left of the pound sign

---

### Step 9.3 Delete your sample Secure Bitcoin Wallet Application HPVS instance

In the output from _Step 9.1_, find your instance named _securewallet_ and copy its CRN value and paste it at the end of the _ibmcloud hpvs instance-delete_ command in the below code cell and then run it.

In [ ]:
! ibmcloud hpvs instance-delete --force    # <- paste your full CRN name to the left of the pound sign

---

### Step 9.4 Delete your sample secure bitcoin wallet application secure image from Docker Hub

While logged in to your Docker Hub account, follow these steps from the Docker Hub web UI in order to delete the image you created in this lab:

1. Click the *Repositories* link in the banner at the top of the page.
2. Click the image that you created in the lab.
3. Click the _Settings_ link.
4. Scroll down to the _Delete Repository_ section and click the _Delete repository_ button.
5. Enter your image name as confirmation–copy and paste is your friend here– for confirmation and click the _Delete_ button.



---

### Step 9.5 Delete the Docker Access Token you created for this lab

Follow [these instructions](https://docs.docker.com/docker-hub/access-tokens/#modify-existing-tokens) to delete the Docker Access Token you created earlier in the lab.



---

### Step 9.6 Remove the ssh key you created for this lab from your GitHub account

Delete the ssh key you added for the lab from your GitHub account by following steps 1-3 of [these instructions](https://docs.github.com/en/github/authenticating-to-github/reviewing-your-ssh-keys), choosing the _Delete_ button for the ssh key you want to get rid of.  If you were an existing GitHub user prior to this lab, be careful not to delete any other ssh keys you may have added for other purposes.

---

### Step 9.7 Remove the GitHub personal access token you created for this lab from your GitHub account

You should still be logged in to GitHub from the previous step, _Step 9.12_.  Click on your picture or avatar for your account and navigate to _Settings->Developer Settings->Personal access tokens_ and delete the personal access token that you created in _Step 2.5_ of this lab.  Ask an instructor for help if you have difficulty deleting this.

---

## You have reached the end of the lab!

Thank you for taking this lab, and please feel free to provide feedback, either by opening a GitHub issue on the lab repo at https://github.com/ibm-wsc/ibm-cloud-hpvs-sbs-byoi-lab or by sending an email directly to the primary author of the lab, Barry Silliman, at "silliman at us dot ibm dot com"  (the email spammers will never figure out my real email address from that, but you will).

---

## Acknowledgements

My esteemed colleagues Jin VanStee and Garrett Woodworth also made significant contributions to this lab, but I take responsibility for any defects or shortcomings in the lab. -  Barry Silliman

## References

This lab is based largely on publicly available material on IBM's website, especially [Tutorial: Using Secure Build Server with a digital wallet](https://cloud.ibm.com/docs/hp-virtual-servers?topic=hp-virtual-servers-tutorial_secure_build_server)

Additional information about using the Secure Build Server CLI can be found [here](https://github.com/ibm-hyper-protect/secure-build-cli)  

Here is the [GitHub repo for the sample secure bitcoin wallet application](https://github.com/IBM/secure-bitcoin-wallet)